#Install yolov8

In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/107.7 GB disk)


In [ ]:
!pip uninstall opencv-python-headless -y
!pip uninstall opencv-python -y
!pip install opencv-python-headless==4.5.5.62
!pip install opencv-python==4.5.5.62


In [ ]:
from IPython import display
display.clear_output()


In [ ]:
import ultralytics
ultralytics.checks()


In [ ]:
!pip install keras
!pip install tensorflow

Mounting drive

In [ ]:
# enter your credentials files path here
creds_file_path = "/home/jovyan/client_secret_1064575295729-8ncp8e1gi66q3rnifh3q8mlgte3rmre2.apps.googleusercontent.com.json"

In [ ]:
!pip3 install google-auth google-api-python-client google-auth-oauthlib

In [ ]:
import io
import os
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Load existing credentials if available
creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

# If credentials are missing or expired, initiate OAuth flow
if not creds or not creds.valid:
    flow = InstalledAppFlow.from_client_secrets_file(
        creds_file_path,
        SCOPES,
        redirect_uri='https://gpu-notebooks.e2enetworks.com/oauth2callback/'
    )
    authorization_url, _ = flow.authorization_url(prompt='consent')
    print(f'Please go to this URL: {authorization_url}')

    authorization_code = input('Enter the authorization code: ')
    flow.fetch_token(code=authorization_code)
    creds = flow.credentials

    # Save the updated credentials for future use
    with open("token.json", "w") as token:
        token.write(creds.to_json())

# Build the Drive API service
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:

#FILE
# Define your query to search for a specific file by name
file_name = "b.mp4"  # Replace with your actual file name
query = f"name = '{file_name}'"

# Execute the query to search for the file
results = drive_service.files().list(q=query, fields="files(id, name, mimeType)").execute()
items = results.get("files", [])

# Check if any files matching the query were found
if not items:
    print(f'No file named "{file_name}" found.')
else:
    print(f'Files named "{file_name}":')
    for item in items:
        print(f"{item['name']} ({item['id']}) - {item['mimeType']}")


In [ ]:
#FILE
# Define the file ID you want to retrieve
file_id = '1--y7mnOkPgWHZ_sIHWTs4VyE9G-rT9Fd'  # Replace with the actual file ID

# Fetch metadata for the specified file ID
file = drive_service.files().get(fileId=file_id, fields="id, name, mimeType").execute()

# Check if the file exists
if file:
    print(f"File found:")
    print(f"{file['name']} ({file['id']}) - {file['mimeType']}")
else:
    print(f'File with ID "{file_id}" not found.')


In [ ]:
file_id = '1--y7mnOkPgWHZ_sIHWTs4VyE9G-rT9Fd'
file_path = '/home/jovyan/b.mp4'

request = drive_service.files().get_media(fileId=file_id)
fh = io.FileIO(file_path, mode='wb')
downloader = MediaIoBaseDownload(fh, request)
done = False

while not done:
    status, done = downloader.next_chunk()

#Install Roboflow Supervision

#prediction of model

In [ ]:
import cv2
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO
from keras import models, layers, optimizers, callbacks
import threading
import queue
import time
import os
import csv

# Load YOLOv8 model from Ultralytics hub
model = YOLO('/content/drive/MyDrive/Road_furniture_weights/content/runs/detect/train/weights/best.pt')  # Update with your model path

# Initialize video capture
video_path = '/content/drive/MyDrive/b.mp4'  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define object tracker class
class ObjectTracker:
    def __init__(self):
        self.objects = {}
        self.next_id = 0
        self.distance_threshold = 50  # Distance threshold for matching objects

    def update(self, box, label):
        center = (box[0], box[1])
        matched_id = None
        for obj_id, obj in self.objects.items():
            dist = np.linalg.norm(np.array(center) - np.array(obj['center']))
            if dist < self.distance_threshold:
                matched_id = obj_id
                break

        if matched_id is None:
            matched_id = self.next_id
            self.next_id += 1
            total_counts[label] += 1  # Only increment when a new object is found

        self.objects[matched_id] = {'center': center, 'label': label}

# Initialize tracking history
track_history = defaultdict(lambda: deque(maxlen=30))
tracker = ObjectTracker()

total_counts = defaultdict(int)

# Object counts and frame tracking
object_counts = defaultdict(list)
frames = []
train_losses = []

# Data collection for training
X_train = []
y_train = []

# Define label mapping
label_mapping = {
    'cautionary': [1, 0, 0],
    'mandatory': [0, 1, 0],
    'informatory': [0, 0, 1]
}

# Define a neural network model for multi-class classification
nn_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(4,)),  # 4 features: x, y, width, height
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')  # Three units for three classes
])
nn_model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Custom callback for real-time training loss visualization
class TrainingLossPlotter(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_losses.append(logs['loss'])

# Initialize the callback
loss_plotter = TrainingLossPlotter()

# Function to train the model in a separate thread
def train_model(X, y, queue):
    history = nn_model.fit(X, y, epochs=50, batch_size=32, callbacks=[loss_plotter], validation_split=0.2)
    queue.put(history)

# CSV file path for object information
csv_file_path = '/content/drive/MyDrive/detected_objects---.csv'
csv_columns = ['object_id', 'timestamp', 'label', 'x', 'y', 'width', 'height']

# Open CSV file for writing
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
    writer.writeheader()

    # Define output video path
    output_video_path = '/content/drive/MyDrive/video_counts---.mp4'  # Replace with your desired output path

    # Explicitly set codec for compatibility
    codec = cv2.VideoWriter_fourcc(*'mp4v')

    # Initialize video writer
    out = cv2.VideoWriter(output_video_path, codec, fps, (frame_width, frame_height))

    # Check if video writer is opened
    if not out.isOpened():
        print("Error: Could not open output video file for writing.")
    else:
        print("Output video file opened successfully.")

        # Start the timer
        start_time = time.time()

        # Data collection and visualization loop
        frame_count = 0
        training_started = False
        training_queue = queue.Queue()
        recorded_objects = set()  # Set to track recorded object IDs

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Perform object detection and tracking
            results = model(frame, conf=0.4, iou=0.4)  # Adjust confidence threshold as needed

            current_counts = defaultdict(int)
            boxes = results[0].boxes.xywh.cpu().numpy()  # Extract bounding boxes (x, y, width, height)
            class_ids = results[0].boxes.cls.cpu().numpy()  # Extract class labels

            for box, class_id in zip(boxes, class_ids):
                label = model.names[int(class_id)]
                tracker.update(box, label)

                obj_id = tracker.next_id - 1  # Use the current next_id
                if obj_id not in recorded_objects:
                    # Calculate timestamp in hr:min:sec format
                    elapsed_seconds = frame_count / fps
                    timestamp = time.strftime('%H:%M:%S', time.gmtime(elapsed_seconds))

                    # Prepare object information for CSV
                    obj_info = {
                        'object_id': obj_id,
                        'timestamp': timestamp,
                        'label': label,
                        'x': box[0],
                        'y': box[1],
                        'width': box[2],
                        'height': box[3]
                    }

                    # Write object info to CSV
                    writer.writerow(obj_info)
                    print(f"Detected object: {obj_info}")  # Debug print statement

                    recorded_objects.add(obj_id)  # Mark object as recorded

                # Collect training data
                if label in label_mapping:
                    X_train.append([box[0], box[1], box[2], box[3]])
                    y_train.append(label_mapping[label])

                # Update tracking history
                track = track_history[class_id]
                track.append((float(box[0]), float(box[1])))  # x, y center point

            frame_with_boxes = results[0].plot()  # Get the frame with bounding boxes

            # Plot the tracks
            for track_id, track in track_history.items():
                points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame_with_boxes, [points], isClosed=False, color=(230, 230, 230), thickness=2)

            # Display total object counts on the frame
            y_pos = 50
            for label, count in total_counts.items():
                cv2.putText(frame_with_boxes, f'{label}: {count}', (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX,  1, (0, 255, 0), 3)
                y_pos += 30

            # Write the frame with boxes to the output video
            out.write(frame_with_boxes)
            print(f"Frame {frame_count} written to output video.")  # Debug print statement

            frame_count += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Wait for training to complete if it has started
        if training_started:
            training_thread.join()
            history = training_queue.get()
            print("Training completed.")

            # Save model weights
            nn_model.save_weights('model.weights.h5')

        # End the timer and print the elapsed time
        end_time = time.time()
        elapsed_time = end_time - start_time
        elapsed_time_str = time.strftime('%H:%M:%S', time.gmtime(elapsed_time))
        print(f"Total time required to run the code: {elapsed_time_str}")

    # Close the CSV file
    print("Closing CSV file.")
    csv_file.close()

# Release video capture
cap.release()
out.release()
cv2.destroyAllWindows()

# Debug: Print final total counts
print("Final total counts:", dict(total_counts))


Video downsampling complete.
